# 案例1.1：流域水资源调查与评价 - 教学版

## 学习目标

1. 掌握水资源评价的基本方法
2. 熟悉频率分析与Pearson-III分布
3. 理解水资源时空分布特征
4. 能够计算不同保证率下的可利用水量

---

## 1. 环境准备

导入必要的库

In [ ]:
import sys
import os

# 添加路径
sys.path.insert(0, '../code')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

# 设置中文字体
plt.rcParams['font.sans-serif'] = ['SimHei', 'Arial Unicode MS', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

print("环境准备完成！")

## 2. 加载数据

加载50年年径流数据和流域参数

In [ ]:
# 加载数据
data_dir = Path('../code/examples/case01_water_resources_assessment/data')

# 年径流数据
annual_runoff = pd.read_csv(data_dir / 'runoff_series.csv')

print(f"数据年份: {annual_runoff['year'].min()} - {annual_runoff['year'].max()}")
print(f"数据点数: {len(annual_runoff)}年")

# 显示前5行
annual_runoff.head()

## 3. 统计分析

计算基本统计量：均值、标准差、变差系数Cv、偏态系数Cs

In [ ]:
from core.utils.statistics import calculate_statistics

runoff_data = annual_runoff['runoff'].values
stats = calculate_statistics(runoff_data)

print("基本统计量:")
print(f"  多年平均径流量: {stats['mean']:.2f} 亿m³")
print(f"  标准差: {stats['std']:.2f} 亿m³")
print(f"  变差系数 Cv: {stats['cv']:.3f}")
print(f"  偏态系数 Cs: {stats['cs']:.3f}")
print(f"  最大值: {stats['max']:.2f} 亿m³")
print(f"  最小值: {stats['min']:.2f} 亿m³")

## 4. 绘制径流序列图

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(annual_runoff['year'], annual_runoff['runoff'], 
         linewidth=1.5, marker='o', markersize=4)
plt.axhline(y=stats['mean'], color='r', linestyle='--', 
            label=f'多年平均 ({stats["mean"]:.2f} 亿m³)')
plt.title('年径流序列', fontsize=14, fontweight='bold')
plt.xlabel('年份', fontsize=12)
plt.ylabel('径流量 (亿m³)', fontsize=12)
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()

## 5. 频率分析

使用Pearson-III型分布进行频率分析

In [ ]:
from core.utils.statistics import frequency_analysis, pearson_iii_distribution

# Pearson-III分布参数
mean, cv, cs = pearson_iii_distribution(runoff_data)
print(f"Pearson-III型分布参数:")
print(f"  均值: {mean:.2f} 亿m³")
print(f"  Cv: {cv:.3f}")
print(f"  Cs: {cs:.3f}")

# 频率分析
probs, design_values = frequency_analysis(
    runoff_data,
    probabilities=[0.5, 0.75, 0.95],
    distribution='pearson3'
)

print(f"\n不同保证率设计值:")
for p, q in zip(probs, design_values):
    print(f"  P={p*100:.0f}%: {q:.2f} 亿m³")

## 6. 绘制频率曲线

In [ ]:
from core.utils.statistics import calculate_exceedance_probability

# 经验点
emp_data, emp_probs = calculate_exceedance_probability(runoff_data)

# 理论曲线
theo_probs = np.array([0.01, 0.05, 0.10, 0.20, 0.50, 0.75, 0.90, 0.95, 0.99])
_, theo_values = frequency_analysis(runoff_data, probabilities=theo_probs)

# 绘图
plt.figure(figsize=(10, 6))
plt.scatter(emp_probs * 100, emp_data, s=50, c='red', marker='o', label='经验点', zorder=3)
plt.plot(theo_probs * 100, theo_values, linewidth=2, c='blue', label='理论曲线', zorder=2)
plt.xscale('log')
plt.xlim([0.1, 99.9])
plt.title('年径流频率曲线 (Pearson-III型)', fontsize=14, fontweight='bold')
plt.xlabel('超过概率 (%)', fontsize=12)
plt.ylabel('年径流量 (亿m³)', fontsize=12)
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()

## 7. 可利用水资源量计算

In [ ]:
# 参数设置
ecological_ratio = 0.20  # 生态基流占比
loss_ratio = 0.05        # 损失率

# 多年平均可利用量
mean_runoff = stats['mean']
available_mean = mean_runoff * (1 - ecological_ratio - loss_ratio)

print(f"多年平均径流量: {mean_runoff:.2f} 亿m³")
print(f"扣除生态基流 ({ecological_ratio*100:.0f}%): {mean_runoff * ecological_ratio:.2f} 亿m³")
print(f"扣除损失 ({loss_ratio*100:.0f}%): {mean_runoff * loss_ratio:.2f} 亿m³")
print(f"可利用水资源量: {available_mean:.2f} 亿m³")

# 不同保证率的可利用量
print(f"\n不同保证率可利用水量:")
for p, q in zip(probs, design_values):
    available_q = q * (1 - ecological_ratio - loss_ratio)
    print(f"  P={p*100:.0f}%: {available_q:.2f} 亿m³")

## 8. 思考题

1. 为什么在水资源评价中常用P=75%和P=95%保证率？
2. 如何确定合理的生态基流？
3. Pearson-III型分布的适用条件是什么？
4. 变差系数Cv的大小说明什么问题？

---

## 总结

本案例完成了：
- ✅ 数据加载与探索
- ✅ 统计分析
- ✅ 频率分析（Pearson-III型）
- ✅ 可利用水量计算
- ✅ 结果可视化

**关键技能**：
- 频率分析方法
- 统计参数估计
- Python数据分析
- 结果可视化